In [1]:
import os
import numpy as np
import pandas as pd

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
from dgl.data import DGLDataset
from dgl.nn import GraphConv
import torch
import torch.nn as nn
import torch.nn.functional as F




from dotenv import dotenv_values

SECRETS = dotenv_values("../envs/graphs.env")

TRAIN_DATA_PATH = SECRETS['TRAIN_DATA_PATH']
TEST_DATA_PATH = SECRETS['TEST_DATA_PATH']
PULL_UP_STRENGTH, PULL_DOWN_STRENGTH = list(map(int, SECRETS['BETA'].split(':')))
NUM_SAMPLES = int(SECRETS['NUM_SAMPLES'])
MAX_LENGTH = int(SECRETS['MAX_LENGTH'])

d:\Thesis\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = [ (np.load(TRAIN_DATA_PATH + f"/train_data/circuit{i}.npz")['adj_list'],  np.load(TRAIN_DATA_PATH + f"/train_data/circuit{i}.npz")['feature_matrix']) for i in range(1, 1000+1) ]

zero_added_train_data = [(train_data[i][0], np.insert(train_data[i][1], -1, np.zeros((1, 1)), axis=1)) for i in range(len(train_data))]
# np.expand_dims(train_data[0][1], axis=0)


In [4]:
df = pd.read_excel("../data/train/finalSim.xlsx")
df['circuit'] = df['circuit'].str.replace('tdlaycircuit', '').astype(int)
df['tdlay'] = df['tdlay'].str.replace('p', '').astype(float)/1000
df.set_index('circuit', inplace=True)
df.sort_index(inplace=True)

In [4]:
# zero_added_train_data[0][1][:, -2] +  list(df['tdlay'])[0]

In [5]:
for i in range(len(zero_added_train_data)):
    zero_added_train_data[i][1][:, -2] += list(df['tdlay'])[i]

In [6]:
zero_added_train_data

[(array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
          16, 17, 18, 19],
         [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
          17, 18, 19, 20]]),
  array([[ 0.   ,  1.   ,  2.   , 22.   ,  0.209,  2.   ],
         [ 1.   ,  1.   ,  2.   , 22.   ,  0.209,  8.   ],
         [ 2.   ,  1.   ,  2.   , 22.   ,  0.209,  4.   ],
         [ 3.   ,  2.   ,  2.   , 22.   ,  0.209, 29.   ],
         [ 4.   ,  3.   ,  2.   , 22.   ,  0.209, 39.   ],
         [ 5.   ,  4.   ,  2.   , 22.   ,  0.209, 47.   ],
         [ 6.   ,  2.   ,  2.   , 22.   ,  0.209, 35.   ],
         [ 7.   ,  0.   ,  2.   , 22.   ,  0.209, 11.   ],
         [ 8.   ,  3.   ,  2.   , 22.   ,  0.209,  3.   ],
         [ 9.   ,  2.   ,  2.   , 22.   ,  0.209, 41.   ],
         [10.   ,  2.   ,  2.   , 22.   ,  0.209, 34.   ],
         [11.   ,  2.   ,  2.   , 22.   ,  0.209,  2.   ],
         [12.   ,  1.   ,  2.   , 22.   ,  0.209, 39.   ],
         [13.   ,  1.   ,  2.

In [7]:
# second_ds = [(
#     zero_added_train_data[i][0], 
#     np.concatenate((zero_added_train_data[i][1][:, :-1],np.random.rand(zero_added_train_data[i][1].shape[0], 4)), axis = 1)
    
#     )]
second_ds = []

for i in range(len(zero_added_train_data)):
    adj_list = zero_added_train_data[i][0]
    feature_matrix = np.concatenate((zero_added_train_data[i][1][:, :-1],np.random.rand(zero_added_train_data[i][1].shape[0], 4)), axis = 1)
    if feature_matrix.shape[0] != zero_added_train_data[i][1].shape[0]:
        print(i, feature_matrix.shape, zero_added_train_data[i][1].shape)
    second_ds.append((adj_list, feature_matrix))

In [9]:
i = 100
temp = dgl.graph((zero_added_train_data[i][0][0], zero_added_train_data[i][0][1]))
temp = dgl.add_self_loop(temp)
temp.ndata['x'] = torch.tensor(zero_added_train_data[i][1], dtype=torch.float32)


In [10]:
i = 100
temp1 = dgl.graph((second_ds[i][0][0], second_ds[i][0][1]))
temp1 = dgl.add_self_loop(temp1)
temp1.ndata['x'] = torch.tensor(second_ds[i][1])
temp1.ndata['y'] = torch.tensor(zero_added_train_data[i][1][:, -1])

In [25]:
graph_classification_dataset, node_classification_dataset = [], []
for i in range(len(zero_added_train_data)):
    temp = dgl.graph((zero_added_train_data[i][0][0], zero_added_train_data[i][0][1]))
    temp = dgl.add_self_loop(temp)
    temp.ndata['x'] = torch.tensor(zero_added_train_data[i][1], dtype=torch.float32)
    graph_classification_dataset.append(temp)

for i in range(len(second_ds)):
    temp1 = dgl.graph((second_ds[i][0][0], second_ds[i][0][1]))
    temp1 = dgl.add_self_loop(temp1)
    temp1.ndata['x'] = torch.tensor(second_ds[i][1], dtype=torch.float32)
    temp1.ndata['y'] = torch.tensor(zero_added_train_data[i][1][:, -1], dtype=torch.float32)
    temp1.ndata['y'] = torch.unsqueeze(temp1.ndata['y'], -1)

    label = temp1.ndata['y'][:, 0].long() - 1
    one_hot_labels = F.one_hot(label, 50)
    temp1.ndata['y'] = one_hot_labels


    node_classification_dataset.append(temp1)

del temp, temp1


In [18]:
labels = node_classification_dataset[0].ndata['y'][:, 0].long() - 1  # Ensure the tensor is of type LongTensor
one_hot_labels = torch.nn.functional.one_hot(labels, num_classes=50)

In [28]:
graph_classification_dataset[0]

Graph(num_nodes=21, num_edges=41,
      ndata_schemes={'x': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={})

In [11]:
# NODE_SPECIFIC_FEATURE_SIZE = 2
# GRAPH_SPECIFIC_FEATURE_SIZE = 3
# K_SIZE = 1
# NOISE_SIZE = 4
# DATASET_SIZE = 1000

# def make_random_graph():
#     num_nodes = np.random.randint(10, 40)
#     src_nodes = np.array([i for i in range(num_nodes-1)])
#     dst_nodes = np.array([i for i in range(1, num_nodes)])
#     random_node_features = np.random.randint(0, 6, (num_nodes, NODE_SPECIFIC_FEATURE_SIZE))
#     random_graph_features = np.random.randint(1, 50, (num_nodes, GRAPH_SPECIFIC_FEATURE_SIZE-1))    
#     random_delay_features = np.random.rand(num_nodes,1)
#     random_K_features = np.random.randint(1, 50, (num_nodes, 1))
#     random_noise_features = np.random.rand(num_nodes, NOISE_SIZE)
#     merged_true_features = np.concatenate((random_node_features, random_graph_features, random_delay_features, random_K_features), axis=1)

#     """
#     [gate_number, type_of_gate, overall_input_cap, overall_output_cap, sizing_of_gate]
#     """


#     merged_false_features = np.concatenate((random_node_features, random_graph_features, random_delay_features, random_noise_features), axis=1)
#     true_graph = dgl.graph((src_nodes, dst_nodes))
#     true_graph = dgl.add_self_loop(true_graph)
#     false_graph = dgl.graph((src_nodes, dst_nodes))
#     false_graph = dgl.add_self_loop(false_graph)
#     true_graph.ndata['x'] = torch.tensor(merged_true_features).float()
#     false_graph.ndata['x'] = torch.tensor(merged_false_features).float()
#     false_graph.ndata['y'] = torch.tensor(random_K_features).float()
#     return true_graph, false_graph


# make_random_graph()

In [29]:
# graph_classification_dataset = []
# node_classification_dataset = []
# for i in range(DATASET_SIZE):
#     true_graph, false_graph = make_random_graph()
#     graph_classification_dataset.append(true_graph)
#     node_classification_dataset.append(false_graph)



class GraphClassificationDataset(DGLDataset):
    def __init__(self, dataset:list, labels:list):
        super().__init__(name='graph_classification_dataset')
        self.graphs = dataset
        self.labels = torch.tensor(labels, dtype=torch.float32)

    
    def process(self):
        return
    
    def __getitem__(self, idx):
        return self.graphs[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.graphs)


true_ds = GraphClassificationDataset(graph_classification_dataset, [1 for i in range(len(graph_classification_dataset))])
        

In [30]:
class NodeClassificationDataset(DGLDataset):
    def __init__(self, dataset:list):
        super().__init__(name='node_classification_dataset')
        self.graphs = dataset
    
    def process(self):
        return
    
    def __getitem__(self, idx):
        return self.graphs[idx]
    
    def __len__(self):
        return len(self.graphs)

node_ds = NodeClassificationDataset(node_classification_dataset)

In [35]:
class NodeClassificationModel(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(NodeClassificationModel, self).__init__()        
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        # Add Softmax layer
        h = F.softmax(h, dim=1)
        return h
    

g = node_ds[0]
model = NodeClassificationModel(g.ndata['x'].shape[1], 10, g.ndata['y'].shape[1])


In [36]:
new_k_values = [model(g, g.ndata['x']) for g in node_ds]

In [40]:
new_k_values[0]

tensor([[3.9264e-04, 8.4313e-03, 2.8917e-03,  ..., 1.7452e-02, 3.0114e-02,
         1.1273e-03],
        [7.0356e-06, 1.2931e-03, 2.0391e-04,  ..., 4.4439e-03, 1.0757e-02,
         4.4588e-05],
        [1.5646e-06, 6.5229e-04, 7.3770e-05,  ..., 2.7786e-03, 6.8251e-03,
         1.4964e-05],
        ...,
        [2.0222e-04, 2.4507e-02, 2.7025e-03,  ..., 3.1820e-02, 2.5257e-02,
         1.5448e-03],
        [2.2738e-04, 2.6485e-02, 2.9347e-03,  ..., 2.8876e-02, 2.4869e-02,
         1.7905e-03],
        [3.6335e-05, 1.7577e-02, 9.7375e-04,  ..., 1.8479e-02, 1.3491e-02,
         6.3185e-04]], grad_fn=<SoftmaxBackward0>)

In [42]:
torch.argmax(new_k_values[2], dim=1)

tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 13, 13, 13, 13,
        13])

In [43]:
def modify_dataset(dataset : GraphClassificationDataset , new_k_values: list):
    new_graphs, new_labels = [], []
    for i in range(len(new_k_values)):
        new_graph = dgl.graph((dataset[i][0].edges()[0], dataset[i][0].edges()[1]))
        new_graph.ndata['x'] = torch.cat((dataset[i][0].ndata['x'][:, :-1], new_k_values[i] ), dim = -1)
        new_graphs.append(new_graph)
        new_labels.append(1-dataset[i][1])

    new_labels = torch.tensor(new_labels)

    new_ds = GraphClassificationDataset(dataset.graphs, torch.tensor(new_labels))
    

    return new_ds



In [44]:
class GraphClassificationModel(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphClassificationModel, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata["h"] = h
        return F.sigmoid(dgl.mean_nodes(g, "h"))
    

graph_model = GraphClassificationModel(true_ds[0][0].ndata['x'].shape[1], 10, 1)


In [45]:
graph_model(true_ds[0][0], true_ds[0][0].ndata['x'])

tensor([[1.0000]], grad_fn=<SigmoidBackward0>)

In [21]:
# from dgl.dataloading import GraphDataLoader
# from torch.utils.data.sampler import SubsetRandomSampler

# num_examples = 1000
# num_train = int(num_examples * 0.8)

# train_sampler = SubsetRandomSampler(torch.arange(num_train))
# test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

# # train_dataloader = GraphDataLoader(
# #     dataset, sampler=train_sampler, batch_size=5, drop_last=False
# # )
# # test_dataloader = GraphDataLoader(
# #     dataset, sampler=test_sampler, batch_size=5, drop_last=False
# # )

In [46]:
generator = NodeClassificationModel(g.ndata['x'].shape[1], 100, g.ndata['y'].shape[1]) # object of class `Generator` to train c-GAN
discriminator = GraphClassificationModel(true_ds[0][0].ndata['x'].shape[1], 100, 1) # object of class `Discriminator` to train c-GAN

criterion = nn.BCELoss()  # Binary crossentropy loss to perform adversarial training of GAN
gen_optimizer = torch.optim.Adam(generator.parameters(), lr=0.008)  # optimizer of `generator` object
disc_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.002)  # optimizer of `discriminator` object


In [47]:
sum(p.numel() for p in generator.parameters())

6050

In [61]:
num_epochs = 7 # Total number of training epochs
discriminator.train()
generator.train()
for epoch in range(num_epochs):
    # for i in range(len(node_ds)):
    discriminator.zero_grad()
    real_loss = 0
    generated_sizings = []
    for i in range(len(true_ds)):
        real_decision = discriminator(true_ds[i][0], true_ds[i][0].ndata['x'])
        real_loss += criterion(real_decision, torch.ones_like(real_decision))

        predicted_k = generator(node_ds[i], node_ds[i].ndata['x'])
        predicted_k = predicted_k.clone().detach()
        predicted_k = torch.unsqueeze(torch.argmax(predicted_k, dim=1), -1)
        generated_sizings.append( predicted_k )
    generated_dataset = modify_dataset(true_ds, generated_sizings)
    
    fake_loss = 0
    for i in range(len(generated_dataset)):
        fake_decision = discriminator(generated_dataset[i][0], generated_dataset[i][0].ndata['x'])
        fake_loss += criterion(fake_decision, torch.zeros_like(fake_decision))

    disc_loss = (real_loss + fake_loss) / 2  
    disc_loss.backward()  
    disc_optimizer.step() 

    generator.zero_grad()
    gen_loss = 0
    for i in range(len(generated_dataset)):
        fake_decision = discriminator(generated_dataset[i][0], generated_dataset[i][0].ndata['x'])
        gen_loss += criterion(fake_decision, torch.ones_like(fake_decision))
    
    gen_loss.backward()
    gen_optimizer.step()

    print(f"Epoch [{epoch}/{num_epochs}], Gen Loss: {gen_loss.item():.4f}, Disc Loss: {disc_loss.item():.4f}")

C:\Users\adith\AppData\Local\Temp\ipykernel_26448\847611993.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  new_ds = GraphClassificationDataset(dataset.graphs, torch.tensor(new_labels))
C:\Users\adith\AppData\Local\Temp\ipykernel_26448\711672344.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.float32)


Epoch [0/7], Gen Loss: 753.5852, Disc Loss: 1346.9358
Epoch [1/7], Gen Loss: 479.2014, Disc Loss: 1417.2003
Epoch [2/7], Gen Loss: 358.5605, Disc Loss: 1493.5922
Epoch [3/7], Gen Loss: 328.1619, Disc Loss: 1479.4691
Epoch [4/7], Gen Loss: 366.8995, Disc Loss: 1360.3971
Epoch [5/7], Gen Loss: 485.0298, Disc Loss: 1165.4084
Epoch [6/7], Gen Loss: 713.4449, Disc Loss: 952.1086
